In [ ]:
import time

from mujoco_py import load_model_from_xml, MjSim, MjViewer, load_model_from_path
import mujoco_py
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('../../')
# import controllers_utils
import tools.rotations as rot
import torch
import time
from D_star_lite import DStar

from KNN_test import FaissKNeighbors

from PRM_tools import collision_check, graph

In [ ]:
q0 = np.zeros(16)
q0[12] = 0.5
s_start =  tuple(q0)
s_goal = (-0.19693, 1.3903904, 1.3273159, 0.35897200000000007,
          -0.06984199999999996, 1.3148996000000002, 1.27591, 0.43,
          0.0, 1.3293476000000002, 1.2510544000000001, 0.55,
          1.1357499,   0.9659528, 1.5200892, 0.6767379)  # goal location

# generate samples
dim = 16
n = 1000
edge_sample_num = 20
k0 = int(np.ceil(np.e * (1 + 1 / dim))) + 2 +5

g = [0, 1, 2, 3, 4]  # palm, index/middle/ring/thumb finger
use_cuda = True

x_obj = np.array([[8.10000000e-02, 0, 1.87000000e-01]])

hand = collision_check(x_obj, use_cuda=use_cuda)

In [ ]:
x_obj = np.array([[0.08, 0, 0.187], [0.08, -0.06, 0.187], [0.08, 0.06, 0.187], [0.11, 0.03, 0.087]])
# x_obj = np.array([[800.10000000e-02, 0, 100.87000000e-01]])  # old

In [ ]:
k0

In [ ]:
x_obj_gpu = torch.Tensor(x_obj).to('cuda:0') if use_cuda else torch.Tensor(x_obj)
hand.x_obj_gpu = x_obj_gpu
graph_16D = graph(dim=dim)
safety_dis = 0.001

# samples
samples = np.random.uniform(hand.nn.hand_bound[0, :], hand.nn.hand_bound[1, :], size=(n, dim))  # (n, dim)
samples = np.concatenate(
    [np.array(s_start).reshape(1, -1), np.array(s_goal).reshape(1, -1), samples])  # (n+2, dim)
knn = FaissKNeighbors(k=k0 + 1)  # knn
knn.fit(samples)
samples_near = knn.predict(samples)[:, 1:, :]  # remove the first one, which is itself
s1 = list(map(tuple, samples))
s2 = [list(map(tuple, samples_near[i, :, :])) for i in range(samples_near.shape[0])]

graph_16D.edges = dict(zip(s1, s2))  # {v1: [a1,a2,a3], v2; [...],...}
d_star = DStar(s_start, s_goal, graph_16D, "euclidean")
start_p = np.repeat(samples, repeats=k0, axis=0)
edge_samples_ = np.linspace(start_p, np.vstack(samples_near),
                            edge_sample_num, axis=1)  # (n*k, edge_sample_num, dim)
edge_samples = np.vstack(edge_samples_)  # (n*k*edge_sample_num, dim)

# pairs = hand.get_dis(edge_samples, gradient=False, sample=edge_sample_num, safety_dis=safety_dis/5, real_dis=True)
hand.q_gpu = torch.Tensor(edge_samples).to('cuda:0') if use_cuda else torch.Tensor(edge_samples)
pairs = hand.collision_hand_obj_SCA(q=None, sample=edge_sample_num, safety_dis=safety_dis)  # do the hand-obj collision and SCA
start_end_p = []
for i in range(len(s2)):
    for j in range(k0):
        start_end_p.append(s1[i] + s2[i][j])

d_star.graph.E = dict(zip(start_end_p, pairs))  # {v1 + v2 : False/True, ...}

result = d_star.ComputePath()
if result:
    path = d_star.extract_path()
    print(len(path))
else:
    path = []
    print(' No feasible path')

In [ ]:
print(path)

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
edge_samples_path.shape

In [ ]:
# check if the path is collision-free
num_tmp = 200
edge_samples_path = np.linspace(np.vstack(path[0:-1]), np.vstack(path[1:]),
                                    num_tmp, axis=1)  # (n*k, edge_sample_num, dim)
edge_samples_path = np.vstack(edge_samples_path)
hand.q_gpu = torch.Tensor(edge_samples_path).to('cuda:0') if use_cuda else torch.Tensor(edge_samples_path)
pairs_path = hand.collision_hand_obj_SCA(sample=num_tmp, safety_dis=safety_dis)

In [ ]:
pairs_path

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free


In [ ]:
pairs_path

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free


In [ ]:
# check if the path is collision-free


In [ ]:
pairs_path

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free


In [ ]:
# check if the path is collision-free
num_tmp = 200
edge_samples_path = np.linspace(np.vstack(path[0:-1]), np.vstack(path[1:]),
                                    num_tmp, axis=1)  # (n*k, edge_sample_num, dim)
hand.q_gpu = torch.Tensor(edge_samples_path).to('cuda:0') if use_cuda else torch.Tensor(edge_samples_path)
pairs_path = hand.collision_hand_obj_SCA(sample=num_tmp, safety_dis=safety_dis)

In [ ]:
pairs_path

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free


In [ ]:
# check if the path is collision-free
num_tmp = 200
edge_samples_path = np.linspace(np.vstack(path[0:-1]), np.vstack(path[1:]),
                                    num_tmp, axis=1)  # (n*k, edge_sample_num, dim)
hand.q_gpu = torch.Tensor(edge_samples_path).to('cuda:0') if use_cuda else torch.Tensor(edge_samples_path)
pairs_path = hand.collision_hand_obj_SCA(sample=num_tmp, safety_dis=safety_dis)

In [ ]:
pairs_path

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free


In [ ]:
# check if the path is collision-free
num_tmp = 200
edge_samples_path = np.linspace(np.vstack(path[0:-1]), np.vstack(path[1:]),
                                    num_tmp, axis=1)  # (n*k, edge_sample_num, dim)
hand.q_gpu = torch.Tensor(edge_samples_path).to('cuda:0') if use_cuda else torch.Tensor(edge_samples_path)
pairs_path = hand.collision_hand_obj_SCA(sample=num_tmp, safety_dis=safety_dis)

In [ ]:
np.sum(pairs) / len(pairs)

In [ ]:
# check if the path is collision-free
